# Practica #6

## Nombre: Christian Zorrilla Gonzalez

## Matrícula: 21-SISN-2-070

## Fecha de entrega: 18 de Octubre de 2023

## Frases e imágenes

Utilizando una interfaz Gradio, crear una aplicación que reciba una entrada de texto y, al ejecutarlo, devuelva una frase relacionada y una imagen.

Ejemplo: si introducimos "guerra", debe mostrar una imagen relacionada con la guerra y una frase sobre este tema.

## Traductor

Utilizando una interfaz Gradio, crear una aplicación que reciba una entrada de texto y lo traduzca a uno de los idiomas disponibles en una lista. La traducción debe realizarse sin agregar nada más. La lista debe contener al menos 5 idiomas.

Ejemplo: si la entrada es "Watashi wa michi to shinri to inochi desu" y se selecciona el idioma español, debe devolver únicamente: "Yo soy el camino, la verdad y la vida", sin agregar nada más."

In [ ]:
import gradio as gr
import requests

# Función para obtener una imagen relacionada con la palabra clave
def obtener_imagen(palabra_clave):
    url = f"https://api.unsplash.com/photos/random?query={palabra_clave}&client_id=YOUR_UNSPLASH_ACCESS_KEY"
    response = requests.get(url)
    data = response.json()
    return data['urls']['regular']

# Función para obtener una frase relacionada con la palabra clave
def obtener_frase(palabra_clave):
    url = f"https://api.example.com/frases?palabra={palabra_clave}"
    response = requests.get(url)
    data = response.json()
    return data['frase']

# Función para ejecutar cuando se haga una predicción en Gradio
def predicciones(palabra_clave):
    imagen_url = obtener_imagen(palabra_clave)
    frase = obtener_frase(palabra_clave)
    return frase, imagen_url

# Crear una interfaz Gradio
iface = gr.Interface(fn=predicciones, inputs="text", outputs=["text", "image"])

# Ejecutar la interfaz de Gradio
iface.launch()

## Traductor


In [1]:
import gradio as gr
import googletrans

# Lista de idiomas disponibles
idiomas = ["español", "inglés", "francés", "portugués", "italiano"]

# Función de traducción
def traducir(texto, idioma):
    traductor = googletrans.Translator()
    return traductor.translate(texto, dest=idioma).text

# Interfaz Gradio
def app():
    entrada = gr.Textbox()
    idioma = gr.Select(idiomas)
    resultado = gr.Output()

    @gr.on_submit
    def handle_submit(texto, idioma):
        resultado.set_text(traducir(texto, idioma))

    return gr.Interface(entrada, idioma, resultado)

app()

AttributeError: module 'gradio' has no attribute 'Select'